# 必須ライブラリのインストール（以下、DataFactoryでコピーしたデータに対して行う作業であり、Functionsを実装する前に検証で行っていた手順。今はFunctionsで同内容を実行するため実施不要）

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

## libraryのインポート

In [ ]:
import json
import logging

from bson import ObjectId
from langchain.docstore.document import Document
from langchain_community.vectorstores.azure_cosmos_db import (
    AzureCosmosDBVectorSearch,
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)
from langchain_openai import AzureOpenAIEmbeddings
from pymongo import MongoClient
from pymongo.collection import Collection


class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return super().default(obj)

## MongoDBのコレクションを取得

In [3]:
def get_all_collections(database_name, collection_name, connection_string):
    # MongoDBクライアントを作成
    mongo_client = MongoClient(connection_string)

    # 指定されたデータベースとコレクションを取得
    db = mongo_client[database_name]
    collection = db[collection_name]

    # コレクション内のすべてのドキュメントを取得して表示
    documents = collection.find()
    print(f"Contents of {collection_name} in {database_name}:")
    for doc in documents:
        print(doc)

## 環境変数の設定

In [ ]:
# データベースとコレクションの名前を指定
# データベース名とコレクション名は、Azure Cosmos DB リソースの名前と一致する必要があります
# 接続文字列は、Azure Cosmos DB リソースの「接続文字列」ページから取得します
database_name = ""
collection_name = ""
connection_string = ""


In [ ]:
# コレクションの中身を表示
get_all_collections(database_name, collection_name, connection_string)

## MongoDBのデータを取得し、埋め込みを生成してAzure Cosmos DBに保存する

In [6]:
async def generate_embeddings_and_add_data(
    documents: list[Document],
    collection: Collection,
    index_name: str,
    embeddings: AzureOpenAIEmbeddings,
) -> AzureCosmosDBVectorSearch:
    # データから埋め込みを生成し、データベースに保存してMongoDB vCoreへの接続を返す
    return await AzureCosmosDBVectorSearch.afrom_documents(
        documents=documents,
        embedding=embeddings,
        collection=collection,
        index_name=index_name,
    )

In [ ]:
# OpenAIの埋め込みモデルとデプロイメント名を設定
openai_embeddings_model = "text-embedding-ada-002"
openai_embeddings_deployment = "text-embedding"

# Azure OpenAIの埋め込みクライアントを初期化
azure_openai_embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    model=openai_embeddings_model,
    azure_deployment=openai_embeddings_deployment,
)

In [ ]:
async def add_data(database_name,collection_name,connection_string) -> None:

    # MongoDBクライアントを作成し、指定されたデータベースとコレクションに接続
    mongo_client: MongoClient = MongoClient(connection_string)
    db = mongo_client[database_name]
    collection = db[collection_name]

    # コレクション内のすべてのドキュメントを取得し、リストに追加
    documents = []
    for idx, item in enumerate(collection.find()):
        documents.append(
            Document(page_content=json.dumps(item), metadata={"source": "mongodb", "seq_num": idx + 1})
        )

    # ドキュメントに対して埋め込みを生成し、Azure Cosmos DBに保存
    vector_store = await generate_embeddings_and_add_data(
        documents=documents,
        collection=collection,
        index_name="CosmicIndex",
        embeddings=azure_openai_embeddings,
    )

    # クエリ条件: metadataキーが存在しないドキュメントを検索
    query = {"metadata": {"$exists": False}}

    # 条件に一致するすべてのドキュメントを削除
    collection.delete_many(query)

    logging.info("✨ Successfully Created the Collection, Embeddings and Added the Data the Collection...")

    # れらの変数の詳細については、以下のリンクを参照してください。https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search
    num_lists = 100
    dimensions = 1536
    similarity_algorithm = CosmosDBSimilarityType.COS
    kind = CosmosDBVectorSearchType.VECTOR_HNSW
    m = 16
    ef_construction = 64

    # コレクションに対してHNSWインデックスを作成
    index_creation = vector_store.create_index(num_lists, dimensions, similarity_algorithm, kind, m, ef_construction)

    logging.info(f"✨ Successfully Created the HNSW Index Over the data...{index_creation}")
    logging.info("✅✅ Done! ✅✅")

In [ ]:
# add_dataの実行
await add_data(database_name,collection_name,connection_string)